# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/classify`(reward model)
- `/start_expert_distribution_record`
- `/stop_expert_distribution_record`
- `/dump_expert_distribution_record`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")

[2025-05-03 21:06:29] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=32318, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1044354026, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1

[2025-05-03 21:06:41] Attention backend not set. Use fa3 backend by default.
[2025-05-03 21:06:41] Init torch distributed begin.


[2025-05-03 21:06:42] Init torch distributed ends. mem usage=0.02 GB
[2025-05-03 21:06:42] Load weight begin. avail mem=78.57 GB


[2025-05-03 21:06:43] Ignore import error when loading sglang.srt.models.llama4. 


[2025-05-03 21:06:44] Using model weights format ['*.safetensors']
[2025-05-03 21:06:44] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


...

[2025-05-03 21:06:45] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768


[2025-05-03 21:06:45] INFO:     Started server process [3620475]
[2025-05-03 21:06:45] INFO:     Waiting for application startup.
[2025-05-03 21:06:45] INFO:     Application startup complete.
[2025-05-03 21:06:45] INFO:     Uvicorn running on http://0.0.0.0:32318 (Press CTRL+C to quit)


[2025-05-03 21:06:46] INFO:     127.0.0.1:41388 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-03 21:06:46] INFO:     127.0.0.1:44216 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-03 21:06:46] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-03 21:06:50] INFO:     127.0.0.1:44232 - "POST /generate HTTP/1.1" 200 OK
[2025-05-03 21:06:50] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-05-03 21:06:51] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-03 21:06:51] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, gen throughput (token/s): 6.26, #queue-req: 0


[2025-05-03 21:06:52] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, gen throughput (token/s): 129.22, #queue-req: 0


[2025-05-03 21:06:52] Decode batch. #running-req: 1, #token: 120, token usage: 0.01, gen throughput (token/s): 139.50, #queue-req: 0
[2025-05-03 21:06:52] INFO:     127.0.0.1:44240 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json.keys() == {"model_path", "is_generation", "tokenizer_path"}

[2025-05-03 21:06:52] INFO:     127.0.0.1:44254 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-05-03 21:06:52] INFO:     127.0.0.1:44266 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-05-03 21:06:52] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-03 21:06:53] INFO:     127.0.0.1:44276 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-05-03 21:06:53] INFO:     127.0.0.1:44278 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-05-03 21:06:53] Cache flushed successfully!
[2025-05-03 21:06:53] INFO:     127.0.0.1:44282 - "POST /flush_cache HTTP/1.1" 200 OK


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-05-03 21:06:53] Start update_weights. Load format=auto
[2025-05-03 21:06:53] Update engine weights online from disk begin. avail mem=41.77 GB
[2025-05-03 21:06:53] Using model weights format ['*.safetensors']


[2025-05-03 21:06:54] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.79it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.78it/s]

[2025-05-03 21:06:54] Update weights end.
[2025-05-03 21:06:54] Cache flushed successfully!
[2025-05-03 21:06:54] INFO:     127.0.0.1:44292 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "qwen/qwen2.5-0.5b-instruct-wrong"
    " (repository not found)."
)

[2025-05-03 21:06:54] Start update_weights. Load format=auto
[2025-05-03 21:06:54] Update engine weights online from disk begin. avail mem=41.77 GB


[2025-05-03 21:06:54] Failed to get weights iterator: qwen/qwen2.5-0.5b-instruct-wrong (repository not found).
[2025-05-03 21:06:54] INFO:     127.0.0.1:44306 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


In [10]:
terminate_process(server_process)

[2025-05-03 21:06:54] Child process unexpectedly failed with an exit code 9. pid=3621050
[2025-05-03 21:06:54] Child process unexpectedly failed with an exit code 9. pid=3620846


## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [11]:
embedding_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-1.5B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-05-03 21:07:04] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-1.5B-instruct', chat_template=None, completion_template=None, is_embedding=True, revision=None, host='0.0.0.0', port=37592, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=376887254, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, bas

[2025-05-03 21:07:18] Downcasting torch.float32 to torch.float16.


[2025-05-03 21:07:18] Overlap scheduler is disabled for embedding models.
[2025-05-03 21:07:18] Downcasting torch.float32 to torch.float16.
[2025-05-03 21:07:18] Attention backend not set. Use fa3 backend by default.
[2025-05-03 21:07:18] Init torch distributed begin.


[2025-05-03 21:07:20] Init torch distributed ends. mem usage=1.02 GB
[2025-05-03 21:07:20] Load weight begin. avail mem=63.65 GB


[2025-05-03 21:07:20] Ignore import error when loading sglang.srt.models.llama4. 


...

Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.12s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.19s/it]

[2025-05-03 21:07:24] Load weight end. type=Qwen2ForCausalLM, dtype=torch.float16, avail mem=58.70 GB, mem usage=4.95 GB.
[2025-05-03 21:07:24] KV Cache is allocated. #tokens: 20480, K size: 0.27 GB, V size: 0.27 GB
[2025-05-03 21:07:24] Memory pool end. avail mem=57.88 GB


[2025-05-03 21:07:25] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-05-03 21:07:25] INFO:     Started server process [3623417]
[2025-05-03 21:07:25] INFO:     Waiting for application startup.
[2025-05-03 21:07:25] INFO:     Application startup complete.
[2025-05-03 21:07:25] INFO:     Uvicorn running on http://0.0.0.0:37592 (Press CTRL+C to quit)


[2025-05-03 21:07:26] INFO:     127.0.0.1:55676 - "GET /v1/models HTTP/1.1" 200 OK
[2025-05-03 21:07:26] INFO:     127.0.0.1:55690 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-03 21:07:26] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-03 21:07:27] INFO:     127.0.0.1:55698 - "POST /encode HTTP/1.1" 200 OK
[2025-05-03 21:07:27] The server is fired up and ready to roll!


In [12]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-1.5B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-05-03 21:07:31] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-03 21:07:31] INFO:     127.0.0.1:49744 - "POST /encode HTTP/1.1" 200 OK


In [13]:
terminate_process(embedding_process)

[2025-05-03 21:07:31] Child process unexpectedly failed with an exit code 9. pid=3623903


## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [14]:
terminate_process(embedding_process)

# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-05-03 21:07:40] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, completion_template=None, is_embedding=True, revision=None, host='0.0.0.0', port=31484, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=163708477, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, downloa

[2025-05-03 21:07:51] Overlap scheduler is disabled for embedding models.
[2025-05-03 21:07:51] Attention backend not set. Use flashinfer backend by default.
[2025-05-03 21:07:51] Init torch distributed begin.


[2025-05-03 21:07:52] Init torch distributed ends. mem usage=0.00 GB
[2025-05-03 21:07:52] Load weight begin. avail mem=74.68 GB


[2025-05-03 21:07:52] Ignore import error when loading sglang.srt.models.llama4. 


[2025-05-03 21:07:54] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-05-03 21:07:57] KV Cache is allocated. #tokens: 20480, K size: 1.25 GB, V size: 1.25 GB
[2025-05-03 21:07:57] Memory pool end. avail mem=57.66 GB
2025-05-03 21:07:57,873 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend


[2025-05-03 21:07:58] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-05-03 21:07:58] INFO:     Started server process [3626211]
[2025-05-03 21:07:58] INFO:     Waiting for application startup.
[2025-05-03 21:07:58] INFO:     Application startup complete.
[2025-05-03 21:07:58] INFO:     Uvicorn running on http://0.0.0.0:31484 (Press CTRL+C to quit)


[2025-05-03 21:07:59] INFO:     127.0.0.1:39116 - "GET /v1/models HTTP/1.1" 200 OK
[2025-05-03 21:07:59] INFO:     127.0.0.1:39124 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-03 21:07:59] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-05-03 21:08:00,837 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90


In [15]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

2025-05-03 21:08:49,365 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-05-03 21:08:49,379 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


2025-05-03 21:09:04,225 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


[2025-05-03 21:09:04] Prefill batch. #new-seq: 2, #new-token: 136, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-03 21:09:04] INFO:     127.0.0.1:39130 - "POST /encode HTTP/1.1" 200 OK
[2025-05-03 21:09:04] The server is fired up and ready to roll!
2025-05-03 21:09:04,437 - INFO - flashinfer.jit: Loading JIT ops: cascade


2025-05-03 21:09:19,160 - INFO - flashinfer.jit: Finished loading JIT ops: cascade
[2025-05-03 21:09:19] INFO:     127.0.0.1:39132 - "POST /classify HTTP/1.1" 200 OK


In [16]:
terminate_process(reward_process)

[2025-05-03 21:09:19] Child process unexpectedly failed with an exit code 9. pid=3626833


## Capture expert selection distribution in MoE models

SGLang Runtime supports recording the number of times an expert is selected in a MoE model run for each expert in the model. This is useful when analyzing the throughput of the model and plan for optimization.

*Note: We only print out the first 10 lines of the csv below for better readability. Please adjust accordingly if you want to analyze the results more deeply.*

In [17]:
expert_record_server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen1.5-MoE-A2.7B --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")

[2025-05-03 21:09:27] server_args=ServerArgs(model_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen1.5-MoE-A2.7B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=39328, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=262396190, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='

[2025-05-03 21:09:38] Attention backend not set. Use flashinfer backend by default.
[2025-05-03 21:09:38] Init torch distributed begin.
[2025-05-03 21:09:38] Init torch distributed ends. mem usage=0.00 GB
[2025-05-03 21:09:38] Load weight begin. avail mem=78.58 GB


[2025-05-03 21:09:38] Ignore import error when loading sglang.srt.models.llama4. 


[2025-05-03 21:09:39] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  12% Completed | 1/8 [00:00<00:05,  1.38it/s]


...

[2025-05-03 21:09:47] INFO:     127.0.0.1:51946 - "GET /v1/models HTTP/1.1" 200 OK
[2025-05-03 21:09:47] INFO:     127.0.0.1:51958 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-03 21:09:47] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-05-03 21:09:49,617 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90


2025-05-03 21:09:49,834 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-05-03 21:09:49,844 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-05-03 21:09:49,951 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


[2025-05-03 21:09:50] Using default MoE kernel config. Performance might be sub-optimal! Config file not found at /public_sglang_ci/runner-l1a-gpu-0/_work/sglang/sglang/python/sglang/srt/layers/moe/fused_moe_triton/configs/E=60,N=1408,device_name=NVIDIA_H100_80GB_HBM3.json, you can create them with https://github.com/sgl-project/sglang/tree/main/benchmark/kernels/fused_moe_triton


2025-05-03 21:09:50,576 - INFO - flashinfer.jit: Loading JIT ops: batch_decode_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False


In [18]:
response = requests.post(f"http://localhost:{port}/start_expert_distribution_record")
print_highlight(response)

url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

response = requests.post(f"http://localhost:{port}/stop_expert_distribution_record")
print_highlight(response)

response = requests.post(f"http://localhost:{port}/dump_expert_distribution_record")
print_highlight(response)

import glob

output_file = glob.glob("expert_distribution_*.csv")[0]
with open(output_file, "r") as f:
    print_highlight("\n| Layer ID | Expert ID | Count |")
    print_highlight("|----------|-----------|--------|")
    next(f)
    for i, line in enumerate(f):
        if i < 9:
            layer_id, expert_id, count = line.strip().split(",")
            print_highlight(f"| {layer_id:8} | {expert_id:9} | {count:6} |")

2025-05-03 21:10:04,817 - INFO - flashinfer.jit: Finished loading JIT ops: batch_decode_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False
[2025-05-03 21:10:04] Resetting expert distribution record...
[2025-05-03 21:10:04] INFO:     127.0.0.1:51988 - "POST /start_expert_distribution_record HTTP/1.1" 200 OK


[2025-05-03 21:10:04] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-05-03 21:10:05] INFO:     127.0.0.1:51972 - "POST /generate HTTP/1.1" 200 OK
[2025-05-03 21:10:05] The server is fired up and ready to roll!


[2025-05-03 21:10:05] INFO:     127.0.0.1:47558 - "POST /generate HTTP/1.1" 200 OK


[2025-05-03 21:10:05] INFO:     127.0.0.1:47562 - "POST /stop_expert_distribution_record HTTP/1.1" 200 OK


[2025-05-03 21:10:05] Resetting expert distribution record...
[2025-05-03 21:10:05] INFO:     127.0.0.1:47564 - "POST /dump_expert_distribution_record HTTP/1.1" 200 OK


In [19]:
terminate_process(expert_record_server_process)

[2025-05-03 21:10:05] Child process unexpectedly failed with an exit code 9. pid=3632656
[2025-05-03 21:10:05] Child process unexpectedly failed with an exit code 9. pid=3632590
